In [ ]:
# Copyright 2025 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Session 처리 분석

In [ ]:
import os
from dotenv import load_dotenv
import argparse

from google.genai import types
from google.adk.runners import Runner

from google.adk.sessions import BaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import VertexAiSessionService

from agent import root_agent

In [ ]:
# 환경 변수 확인.

# from dotenv import dotenv_values
# config = dotenv_values("../../.env")
# for key, value in config.items():
#     print(f"{key}: {value}")

In [ ]:
APP_NAME = "AI_assistant"
USER_ID = "Forusone"
SESSION_ID = "forus_sess_001"

# Session 유형은 크게 세 가지(in_memory, database, agent_engine)가 있으나 여기서는 두가지만 처리하기 합니다.
SESSION_TYPE = "database" # in_memory 

session_service = None

if SESSION_TYPE == "in_memory":
    session_service = InMemorySessionService()

elif SESSION_TYPE == "database":
    db_url = "sqlite+aiosqlite:///./adk_session.db"
    session_service = DatabaseSessionService(db_url=db_url)

else:
    raise ValueError("잘못된 세션 유형입니다. 'in_memory', 'database' 중 하나를 선택하세요.")

In [ ]:
# 주어진 app_name 및 user_id에 대해 세션 서비스에서 기존 세션을 쿼리합니다.
existing_sessions = await session_service.list_sessions(
    app_name=APP_NAME,
    user_id=USER_ID,
)

print(f"현재 존재하는 세션 ID 들\n")
for session in existing_sessions.sessions:
    print(f"세션 ID :{session.id}")


In [ ]:
# 사용자에 의해서 주어진 SESSION_ID 를 가진 세션이 존재하는지 확인합니다. session_id가 None이면 결과는 None이 됩니다.
existing_session_id = next((session.id for session in existing_sessions.sessions if session.id == SESSION_ID), None)

print(f"사용자에 의해 주어진 세션 {SESSION_ID}가 존재하는지 확인: {existing_session_id}")   

In [ ]:
# 기존 세션이 없으면 새 세션을 생성합니다.
if existing_session_id is None:
    
    # 다른 세션 서비스의 경우 session_id를 create_session에 전달하여 특정 세션 ID를 설정할 수 있습니다.
    new_session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        state=None,
        session_id=SESSION_ID )

    new_session_id = new_session.id
    print(f"새 세션 생성됨: {new_session_id}")

else:
    print(f"기존 세션 사용: {existing_session_id}")

SESSION_ID = existing_session_id or new_session_id

In [ ]:
# 에이전트 러너 초기화.
runner = Runner(agent=root_agent,
                app_name=APP_NAME,
                session_service=session_service)


In [ ]:
print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
    print("\n👤 User :   ")

    # 사용자 입력에서 쿼리 가져오기
    query = input("")
    if query.strip().lower() in ["exit", "quit"]:
        break
    else:
        print(query)
    
    content = types.Content(role='user', parts=[types.Part(text=query)])

    # 에이전트 러너를 사용하여 에이전트 이벤트를 비동기적으로 실행합니다.
    # 세션을 유지하기 위해 user_id와 session_id를 제공합니다. 대화당 하나의 세션만 사용됩니다.
    # 새 세션 생성을 강제하려면 session_id=None을 전달하세요.
    events = runner.run_async(user_id=USER_ID,
                            session_id=SESSION_ID,
                            new_message=content, )

    async for event in events:

        # app_name, user_id, session_id를 사용하여 세션 서비스에서 세션을 가져옵니다.
        session  = await session_service.get_session(app_name=APP_NAME,
                                                        user_id=USER_ID,
                                                        session_id=SESSION_ID)

        print(f"--- 세션 속성 검사 ---")
        print(f"ID (`id`):                {session.id}")
        print(f"애플리케이션 이름 (`app_name`): {session.app_name}")
        print(f"사용자 ID (`user_id`):         {session.user_id}")
        print(f"상태 (`state`):           {session.state}") # 참고: 여기서는 초기 상태만 표시됩니다.
        print(f"이벤트 (`events`):         {session.events}") # 초기에는 비어 있음
        print(f"마지막 업데이트 (`last_update_time`): {session.last_update_time:.2f}")
        print(f"---------------------------------")

        if event.is_final_response():
            final_response = event.content.parts[0].text            
            print("\n 🤖 AI Assistant: " + final_response)
        